In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movie=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movie.head(1)

In [ ]:
movie.shape

In [ ]:
credits.head(1)

In [ ]:
credits.shape

In [ ]:
movie=movie.merge(credits,on='title')

In [ ]:
movie=movie[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movie.head(1)

In [ ]:
movie.isnull().sum()

In [ ]:
movie.dropna(inplace=True)

In [ ]:
movie.shape

In [ ]:
movie.duplicated().sum()

In [ ]:
import ast

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movie['genres']=movie['genres'].apply(convert)

In [ ]:
movie['keywords']=movie['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movie['cast']=movie['cast'].apply(convert3)

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movie['crew']=movie['crew'].apply(fetch_director)

In [ ]:
movie['overview']=movie['overview'].apply(lambda x: x.split())

In [ ]:
movie['genres']=movie['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movie['cast']=movie['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movie['crew']=movie['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [ ]:
movie.head(1)

In [ ]:
movie['tags']=movie['overview'] + movie['genres'] + movie['keywords'] + movie['cast'] + movie['crew']

In [ ]:
new_df=movie[['movie_id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x : ' '.join(x))

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df

In [ ]:
new_df['tags'][0]

# stemming

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

# converting in vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
len(cv.get_feature_names())

# cosine similarity to finding distance

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
def recommend(movie):
    
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
new_df[new_df['title']=='Newlyweds'].index[0]

In [ ]:
recommend('The Dark Knight Rises')

In [ ]:
new_df

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))